### Daten importieren

In [4]:
import pandas as pd
import plotly.express as px
import numpy as np

### Daten auslesen

In [5]:
df = pd.read_csv('flower_data.csv')

dict = {'name': 'Name',
        'height (cm)': 'Höhe',
        'longevity (years)': 'Lebenserwartung',
        'place of origin': 'Herkunftskontinent',
        'color': 'Farbe',
        'climate': 'Klima',
        'cut flowers': 'Geschnittene Blume',
        'perfumes': 'Parfum',
        'medicine': 'Medizin',
        'average number of petals': 'Durchschnittliche Anzhal Blätter'}

df.rename(columns=dict, inplace=True)

display(df)

,Name,Höhe,Lebenserwartung,Herkunftskontinent,Farbe,Klima,Geschnittene Blume,Parfum,Medizin,Durchschnittliche Anzhal Blätter
0,Rose,60-150,1-2,"Europe, Asia","Red, pink, white",Temperate,1,1,1,5-9
1,Tulip,20-70,2-3,Central Asia,"Red, pink, white",Temperate,1,0,1,3-7
2,Daisy,15-60,1-2,"Europe, Asia","White, yellow",Temperate,1,0,1,10-20
3,Sunflower,50-300,1-2,North and South America,"Yellow, orange",Temperate to tropical,1,0,1,10-20
4,Orchid,20-100,5-10,Worldwide,"Purple, pink, white",Tropical,1,1,1,3-50
...,...,...,...,...,...,...,...,...,...,...
67,Hyacinth,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10
68,Iris reticulata,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10
69,Chionodoxa,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10
70,Anemone,10-20,1-2,"Europe, Asia","Purple, pink, white",Temperate,1,0,1,5-10


### Daten filtern

In [6]:
# Split the "Herkunftskontinent" column
df["Herkunftskontinent"] = df["Herkunftskontinent"].str.split(", ")

# Explode the DataFrame
df = df.explode("Herkunftskontinent")

# Filter the DataFrame for "Asia" and "Europe"
df_filtered = df.loc[df["Herkunftskontinent"].isin(["Asia", "Europe"])]

# Print the resulting DataFrame
df_filtered.head()

,Name,Höhe,Lebenserwartung,Herkunftskontinent,Farbe,Klima,Geschnittene Blume,Parfum,Medizin,Durchschnittliche Anzhal Blätter
0,Rose,60-150,1-2,Europe,"Red, pink, white",Temperate,1,1,1,5-9
0,Rose,60-150,1-2,Asia,"Red, pink, white",Temperate,1,1,1,5-9
2,Daisy,15-60,1-2,Europe,"White, yellow",Temperate,1,0,1,10-20
2,Daisy,15-60,1-2,Asia,"White, yellow",Temperate,1,0,1,10-20
5,Jasmine,60-180,5-10,Asia,"White, yellow",Tropical,1,1,1,5-9


### Kreis-Diagramm: Herkunftskontinent

In [7]:
font_family = 'Arial'

df["Herkunftskontinent"] = df["Herkunftskontinent"].str.split(", ")
df = df.explode("Herkunftskontinent")

#counts all instances of continents
origin_counts = df["Herkunftskontinent"].value_counts()

#plotly pastell color palette
colors = px.colors.qualitative.Pastel

#creates chart
fig = px.pie(values=origin_counts, names=origin_counts.index, title="Herkunftskontinent der Blumen", hole=0.5,
             color_discrete_sequence=colors)

#layout adjustments
fig.update_layout(title={'font': {'family': font_family, 'size': 24}, 'x': 0.5, 'y': 0.9},
                  font={'family': font_family, 'size': 12},
                  legend_title='Kontinente',
                  legend={'x': 0.75,  'y': 0.5})

fig.show()

### Kuchen-Diagramm: Europa vs Asien

In [8]:
font_family = 'Arial'

asia_europe = df_filtered["Herkunftskontinent"].value_counts()

#plotly pastell color palette
colors = px.colors.qualitative.Pastel

#creates pie chart form asia europe series
fig = px.pie(values=asia_europe, names=asia_europe.index, title="Asia vs Europe")
                                                                                      
#add dropdown

#layout adjustments
fig.update_layout(
    title={'font': {'family': font_family, 'size': 24}, 'x': 0.5, 'y': 0.9},
    font={'family': font_family, 'size': 12},
    legend_title='Gewählte Kontinente',
    legend={'x': 0.7,  'y': 0.5}
    )

fig.show()

In [9]:
### Aufgabe 5a) and 5b)

In [10]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df = pd.DataFrame({'y': np.random.normal(loc=0, scale=10, size=1000),
                   'x': np.random.normal(loc=10, scale=2, size=1000)})

app.layout = html.Div([html.H1("Dashboard 2"),
    # Aufgabe 5a
    dbc.Row([
        dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color', multi=False)], width=6),
        dbc.Col([dcc.RangeSlider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), value=[math.floor(df['y'].min()), math.ceil(df['y'].max())], id="my-range-slider")], width=6)
        ]),

    # Aufgabe 5b
    dbc.Row([
        dbc.Col([dcc.Slider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), id="min_value")], width=6),
        dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color_2', multi=False)], width=6),
    ]),

    # Darstellung der Diagramme
    dbc.Row([dbc.Col([dcc.Graph(id="graph_1")], width=6),
             dbc.Col([dcc.Graph(id="graph_2")], width=6)
    ])], className="m-4")

# Diagramm 1 --> Aufgabe 5a und 5b
@app.callback(Output("graph_1", "figure"), Input("color", "value"), Input("min_value", "value"))

def update_graph_1(dropdown_value_color, min_value):
    dff = df[df['y']> min_value]
    fig = px.histogram(dff, x="y", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout()
    return fig


# Diagramm 2 --> Aufgabe 5a
@app.callback(Output("graph_2", "figure"), [Input("my-range-slider", "value"), Input("color_2", "value")])

def update_graph_2(range_values, color):
    dff = df[(df['y'] > range_values[0]) & (df['y'] < range_values[1])]
    fig = px.scatter(dff, x='x', y='y', color_discrete_sequence=[color])
    fig.update_layout()
    return fig


if __name__ == '__main__':
    app.run_server(debug=False, port=8000)

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit


In [ ]:
### Aufgabe 6 a) und b)

In [11]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Random Daten --> Tab 1
df_random = pd.DataFrame({'y': np.random.normal(loc=0, scale=10, size=1000),
                          'x': np.random.normal(loc=10, scale=2, size=1000)})

# Daten --> Tab 2 
df_testing = pd.DataFrame({
 'x': [1, 2, 3, 4, 5],
 'Monstera': [1, 4, 2, 3, 7],
 'Strilezia': [2, 3, 1, 5, 6],
 'Wüstenrose': [3, 5, 4, 2, 8],
 'Juwelorchidee': [4, 2, 6, 1, 9]
})

# Unser Datensatz --> Tab 3
df_own = pd.read_csv('flower_data.csv')

dict = {'name': 'Name',
        'height (cm)': 'Höhe',
        'longevity (years)': 'Lebenserwartung',
        'place of origin': 'Herkunftskontinent',
        'color': 'Farbe',
        'climate': 'Klima',
        'cut flowers': 'Geschnittene Blume',
        'perfumes': 'Parfum',
        'medicine': 'Medizin',
        'average number of petals': 'Durchschnittliche Anzhal Blätter'}

df_own.rename(columns=dict, inplace=True)


app.layout = html.Div([
    html.Div([html.H1("Dashboard 3")], className="header"),
    html.Div([
        dcc.Tabs(id="tabs", children=[

            # Tab 1 (vorgegeben)
            dcc.Tab(label='Tab One', id="tab_1_graphs", children=[
                html.Div([
                    dbc.Row([
                        dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color', multi=False)], width=6),
                        dbc.Col([dcc.Slider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), id="min_value")], width=6)
                    ]),
                    dbc.Row([
                        dbc.Col([dcc.Graph(id="graph_1")], width=6),
                        dbc.Col([dcc.Graph(id="graph_2")], width=6)
                    ])
                ], className="tab_content")
            ]),

            # Tab 2 --> Aufgabe 6a
            dcc.Tab(label='Tab Two', id="tab_2_graphs", children=[
                html.Div([
                    dbc.Row([
                        dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color', multi=False)], width=6),
                        dbc.Col(dcc.Graph(id="graph_3"), width=6)
            ])
                ])
            ]),

            # Tab 3 --> für Aufgabe 6b
            dcc.Tab(label='Tab Three', id="tab_3_graphs", children=[
                html.Div([
                    dbc.Row([
                        dbc.Col([dcc.Graph(id="graph_4")], width=12)
                    ]),
                    dbc.Row([
                        dbc.Col([dcc.Graph(id="graph_5")], width=12),
                    ])
                ])
            ])
        ])
    ])
])

# vorgegebener Code --> Tab 1
@app.callback(Output("graph_1", "figure"), Input("color", "value"))
def update_graph_1(dropdown_value_color):
    fig = px.histogram(df, x="y", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout(template="plotly_white")
    return fig

# vorgegebener Code --> Tab 1
@app.callback(Output("graph_2", "figure"), Input("min_value", "value"))
def update_graph_2(min_value):
    if min_value:
        dff = df[df['y'] > min_value]
    else:
        dff = df
    fig = px.scatter(dff, x='x', y='y')
    fig.update_layout(template="plotly_white")
    return fig

# Tab 2 --> Aufgabe 6a
@app.callback(Output('graph_3', 'figure'), Input('color', 'value'))

def update_graph_3(color):
    fig = px.line(df_testing, x='x', y=['Monstera', 'Strilezia', 'Wüstenrose', 'Juwelorchidee'], color_discrete_sequence=[color])
    fig.update_layout(template="plotly_white")
    fig.update_xaxes(title="Anzahl Stunden gespielter klassischer Musik für die Pflanze")   
    fig.update_yaxes(title="Anzahl Lebensjahre der Pflanze")
    return fig


# Tab 3 --> Aufgabe 6b
@app.callback(Output("graph_4", "figure"), Input("min_value", "value"))
def update_graph_4(min_value):
    if min_value:
        dff = df_own[df_own['Höhe'] > min_value]
    else:
        dff = df_own
    
    # Daten für das Balkendiagramm
    count_by_color = dff['Farbe'].value_counts()
    colors = count_by_color.index.tolist()
    counts = count_by_color.tolist()
    
    # Erstellen des Balkendiagramms
    fig = px.bar(x=colors, y=counts, title="Anzahl der Blumen für jede Farbe bzw. Farbkombination")
    fig.update_layout(template="plotly_white")
    fig.update_xaxes(title="Farbe")
    fig.update_yaxes(title="Anzahl der Blumen")
    return fig

# Tab 3 --> Aufgabe 6b
@app.callback(Output("graph_5", "figure"), Input("min_value", "value"))
def update_graph_5(min_value):
    if min_value:
        dff = df_own[df_own['Höhe'] > min_value]
    else:
        dff = df_own
    fig = px.scatter(dff, x='Durchschnittliche Anzhal Blätter', y="Höhe", title="Durchschnittliche Anzahl der Blätter in Bezug auf die Höhe der Blume")
    return fig

if __name__ == '__main__':
    app.run_server(debug=False, port=8010)

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8010
Press CTRL+C to quit


In [ ]:
### Aufgabe 7 a) und b)

In [4]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc
from sklearn.datasets import make_blobs

# new: more than one plot in a callback
# new: one plot as an input for another plot
# new: plotly go object

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df = pd.DataFrame({'y': np.random.normal(loc=0, scale=10, size=1000),
                   'x': np.random.normal(loc=10, scale=2, size=1000)})

# define cluster colors

COLORS = {'0': "red", '1': "blue", '2': "grey"}

X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=0)

cluster_df = pd.DataFrame(data=X, gcolumns=["X", "Y"])
cluster_df['cluster'] = [str(i) for i in y]

app.layout = html.Div([html.Div([html.H1("Dashboard 4")], className="header"), html.Div([dcc.Tabs(id="tabs", children=[
                 dcc.Tab(label='Tab One', id="tab_1_graphs", children=[html.Div([
                      dbc.Row([dbc.Col([dcc.Dropdown(options=['red','green','blue'], value='red', id='color', multi=False)], width=6),
                               dbc.Col([dcc.Slider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), id="min_value")], width=6)]),
                      dbc.Row([dbc.Col([dcc.Graph(id="graph_1")], width=6),
                               dbc.Col([dcc.Graph(id="graph_2")], width=6)])], className="tab_content"),]),
                 dcc.Tab(label='Tab Two', id="tab_2_graphs", children=[html.Div([
                      dbc.Row([dbc.Col([dcc.Graph(id="graph_3")], width=8),
                               dbc.Col([dcc.Graph(id="graph_4")], width=4)])], className="tab_content")]),])], className="content")])

@app.callback(Output("graph_1", "figure"), Input("color", "value"))

def update_graph_1(dropdown_value_color):
    fig = px.histogram(df, x="y", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout(template="plotly_white")
    return fig

@app.callback(Output("graph_2", "figure"), Input("min_value", "value"))

def update_graph_2(min_value):
    if min_value:
        dff = df[df['y'] > min_value]
    else:
        dff = df
    fig = px.scatter(dff, x='x', y='y')
    fig.update_layout(template="plotly_white")
    return fig

@app.callback(Output("graph_3", "figure"), Output("graph_4", "figure"), Input("graph_3", "relayoutData"))

def update_graph_3_and_4(selected_data):
    if selected_data is None or (isinstance(selected_data, dict) and 'xaxis.range[0]' not in selected_data):
        cluster_dff = cluster_df
    else:
        cluster_dff = cluster_df[(cluster_df['X'] >= selected_data.get('xaxis.range[0]')) &
                                 (cluster_df['X'] <= selected_data.get('xaxis.range[1]')) &
                                 (cluster_df['Y'] >= selected_data.get('yaxis.range[0]')) &
                                 (cluster_df['Y'] <= selected_data.get('yaxis.range[1]'))]

    fig3 = px.scatter(cluster_dff, x="X", y="Y", color="cluster", color_discrete_map=COLORS, category_orders= {"cluster": ["0", "1", "2"]}, height=750)
    fig3.update_layout(template="plotly_white", coloraxis_showscale=False)
    fig3.update_traces(marker=dict(size=8))

    group_counts = cluster_dff[['cluster', 'X']].groupby('cluster').count()

    fig4 = go.Figure(data=[go.Bar(x=group_counts.index, y=group_counts['X'], marker_color= [COLORS.get(i) for i in group_counts.index])])
    fig4.update_layout(height=750, template="plotly_white", title="<b>Counts per cluster</b>", xaxis_title="cluster", title_font_size= 25)
    return fig3, fig4

if __name__ == '__main__':
    app.run_server(debug=True, port=8012)

TypeError: DataFrame.__init__() got an unexpected keyword argument 'gcolumns'